# Pipeline in Jupyter Notebook

In [1]:
from src.data_preprocessing import load_data, preprocess_data, save_processed_data

# Load and preprocess data
raw_data_path = "data/raw/gender-classification.csv"
processed_data_path = "data/processed"
processed_data_filename = "processed_data.csv"
text_column = "text"
label_column = "gender"

# Load raw data (csv file)
raw_data = load_data(raw_data_path)

# Preprocess data
preprocessed_data = preprocess_data(raw_data, text_column, label_column)

# Save preprocessed data to a csv file
save_processed_data(
    preprocessed_data, processed_data_path, processed_data_filename
)

[nltk_data] Downloading package punkt_tab to venv/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package stopwords to venv/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Data loaded successfully from: data/raw/gender-classification.csv
Data preprocessing completed successfully!
Processed data saved at: data/processed/processed_data.csv


In [2]:
# Load preprocessed data
df = load_data(f"{processed_data_path}/{processed_data_filename}")

Data loaded successfully from: data/processed/processed_data.csv


In [ ]:
# Data Augmentation
from src.data_augmentation import augment_text, add_augmented_data_to_dataframe, save_augmented_data

# Add augmented data to the preprocessed data
augmented_data = add_augmented_data_to_dataframe(df, text_column, label_column)
save_augmented_data(augmented_data, processed_data_path, "augmented_data.csv")

[nltk_data] Downloading package wordnet to venv/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to venv/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


Augmented data saved at: data/processed/augmented_data.csv


In [ ]:
# Load augmented data
df = load_data(f"{processed_data_path}/augmented_data.csv")